In [ ]:
using Pkg
if isfile("../Project.toml") && isfile("../Manifest.toml")
    Pkg.activate("..");
    ENV["PYTHON"] = "python3";
end
 
#Pkg.build("PyCall")
using JuliaProbo
using Seaborn: heatmap, savefig
gr()

In [ ]:
function init_state_transition_probs(pe::PolicyEvaluator, time_interval::Float64, sampling_num::Int64)
    reso = pe.reso
    indices = pe.indices
    policy = pe.policy_
    actions = Set{Vector{Float64}}()
    index_nums = pe.index_nums
    pose_min = pe.pose_min
    for index in indices
        push!(actions, policy[index..., :])
    end
    
    dxs = collect(range(0.001, reso[1]*0.999, length=sampling_num))
    dys = collect(range(0.001, reso[2]*0.999, length=sampling_num))
    dθs = collect(range(0.001, reso[3]*0.999, length=sampling_num))
    # key is (v::Float64, ω::Float64, θ_index::Int64,)
    # value is vector of ([tran_x_id, tran_y_id, tran_z_id], prob,)
    tmp = Dict{ Tuple{Float64, Float64, Int64}, Vector{Tuple{Vector{Int64}, Float64}} }()
    for a in actions
        for i_θ in 1:index_nums[3]
            transitions = Vector{Vector{Int64}}(undef, 0)
            for dx in dxs
                for dy in dys
                    for dθ in dθs
                        before = [dx, dy, dθ+(i_θ-1)*reso[3]] .+ pose_min
                        before_index = [1, 1, i_θ]
                        after = state_transition(before, a[1], a[2], time_interval)
                        after_index_ = (after - pose_min) ./ reso
                        after_index = map(x -> convert(Int64, floor(x) + 1), after_index_)
                        push!(transitions, after_index-before_index)
                    end
                end
            end
            elems = unique(transitions)
            counts = [count(x -> x==elem, transitions) for elem in elems]
            probs = [c / (sampling_num)^3 for c in counts]
            tmp[(a[1], a[2], i_θ,)] = [iter for iter in zip(elems, probs)]
        end
    end
    return tmp
end

In [ ]:
pe = PolicyEvaluator([0.1, 0.1, pi/20], Goal(-3.0, -3.0));
init_value(pe)
init_policy(pe)

In [ ]:
x = init_state_transition_probs(pe, 0.1, 10)
for (k, v) in x
    println("$(k) => $(v)")
    println("")
end